In [2]:
# Cell 1: Load Modules
import os
import copy
# Note that pandas = pd
import pandas as pd

In [2]:
# Cell 2: Canvas
# A) ADD basics to map
# SHAPEFILES
# Add below shapefiles to a new map - located in github_data dir
#   1) FH_Perimeter
#   2) contours_100m_4corners

# ZOOM INTO FIRE
# zoom into vicinity of Four Corners Fire by zooming into contours_100m_4corners

# FORMAT symbology if you are feeling inspired
# fire perimeter just lines, no fill
# contours graduated

In [3]:
# Cell 3: Paths
# B) input path to your unzipped git directory
user_git_dir = r'C:\Users\uhlmann\Documents\urisa_conference_2022\notebooks_tutorial'

In [1]:
# Cell 4: Load Pandas DataFrame
# REPLACE below path/to/data/inventory.csv to location in your directory
fp_csv = os.path.join(user_git_dir, r'four_corners_morels_inv.csv')
# read csv into Pandas dataframe
df = pd.read_csv(fp_csv, index_col = 'ITEM')
# Print df to output - SEE!! pretty cool
df

In [8]:
# Cell 5: Incorrect File Path
# FYI, we need to fix this path/to/RoadCore_FS in the dataframe before proceeding
df.loc['RoadCore_FS', 'DATA_LOCATION']

'C:\\Users\\uhlmann\\Documents\\urisa_conference_2022\\notebooks_tutorial\\github_data\\RoadCore_FS.shp'

In [3]:
# Cell 6: Fix Path the hard way 
# OR - you could also skip this cell and simply replace DATA_LOCATION for RoadCore_FS in csv 
# to the path/to/your/notebooks_tutorial/git_data/RoadCore_FS.shp
# and rerun cell 1) to reload the dataframe (df)

zachs_git_dir = r'C:\Users\uhlmann\Documents\urisa_conference_2022\notebooks_tutorial'

# replace paths in dataframe
indices = copy.copy(df.index)
fp_orig = df.loc[indices, 'DATA_LOCATION']
fp_fixed = []
for fp in fp_orig:
    try:
        fp_fixed.append(fp.replace(zachs_git_dir, user_git_dir))
    # four_corners_fire_perim has no DATA_LOCATION; except clause will handle
    except AttributeError:
        fp_fixed.append('')
df['DATA_LOCATION'] = fp_fixed

# SEE! now the path/to/RoadCore_FS.shp is corrected for your file system
# If it's NOT, check the path in Cell 3 again.  Make sure you use forward slashes (\)
df.loc['RoadCore_FS', 'DATA_LOCATION']

In [4]:
# Cell 7: Dictionary
# Dictionary for code readability later
# REPLACE path/to/gdb.gdb with path/to/gdb.gdb on your directory/computer:
# key:val = 'urisa_presentation_gdb': 'path/to/gdb'
fp_gdb = os.path.join(user_git_dir, 'four_corners_morels/four_corners_morels.gdb')
path_dict = {'four_corners_morels_gdb':fp_gdb}
# Print dictionary for inspection
path_dict

In [12]:
# Cell 8: Arguments for copy fc
# copy feature from TOC (table of contents)
# NOTE: select 4 corners fire from att table!
feat_in = 'FH_Perimeter'
index = 'four_corners_fire_perim'
gdb_target_str = df.loc[index, 'MOVE_LOCATION']
# use key: gdb_target_str to get path/to/gdb value from dictionary
gdb_target = path_dict[gdb_target_str]
dset_name = df.loc[index, 'MOVE_LOCATION_DSET']
dir_target = os.path.join(gdb_target, dset_name)
dir_target
fname = copy.copy(index)

In [17]:
# Cell 9: CREATE DATASETs - agency, analysis
# Open and enter Notebook titled "basics" and create datasets by running all cells in order
# Follow instructions therein as we will create TWO datasets: Agency and Analysis

In [5]:
# Cell 10: Create new FC
# Create new FC in agency dset with JUST Four Corners fire feature (not all ~140 fire boundaries)
# IMPORTANT: From FH_Perimeter (feat_in) feature class, select 4 corners fire from attribute table(!!)
# prior to running final line --> arcpy.FeatureClass...
# Note - feature = FID 143

print('FC TO COPY: ', feat_in)
print('COPY LOCATION: ', dir_target)
print('FEATURE CLASS NAME OUT: ', fname)

# # b) Once arguments are confirmed, uncomment below line and run.  
# # To uncomment press "Ctl + /" or simply erase the #
arcpy.FeatureClassToFeatureClass_conversion(feat_in, dir_target, fname)

In [ ]:
# Cell 11: Clean up map
# Remove FH_Perimeter from TOC

In [17]:
# Cell 12: Set values to DataFrame
# since we copied the fc to our project gdb, let's add the updated path to dataframe
df.loc[index, 'DATA_LOCATION'] = os.path.join(dir_target, fname)

In [18]:
# Cell 13: Arguments for copy Roads FC
# copy features from file path in DataFrame to pro project gdb
index = 'RoadCore_FS'
feat_in = df.loc[index,'DATA_LOCATION']
gdb_target_str = df.loc[index, 'MOVE_LOCATION']
gdb_target = path_dict[gdb_target_str]
dset_name = df.loc[index, 'MOVE_LOCATION_DSET']
dir_target = os.path.join(gdb_target, dset_name)
dir_target
fname = copy.copy(index)

In [6]:
# Cell 14: Copy roads feature class to Pro gdb
print('FC TO COPY: ', feat_in)
print('COPY LOCATION: ', dir_target)
print('FEATURE CLASS NAME OUT: ', fname)

# # b) Once arguments are confirmed, uncomment and run
arcpy.FeatureClassToFeatureClass_conversion(feat_in, dir_target, fname)

In [20]:
# Cell 15: Set values to DataFrame
# save data location original to dataframe
df.loc[index,'DATA_LOCATION_ORIGINAL'] = df.loc[index, 'DATA_LOCATION']
# since we copied the fc to our project gdb, let's add the updated path to dataframe
df.loc[index,'DATA_LOCATION'] = os.path.join(dir_target,fname)

In [7]:
# Cell 16) Clip contour polygons to fire boundary
fp_in = 'contours_100m_4corners'
fp_clip = 'four_corners_fire_perim'
index = 'contours_100m_4corners_clipped'
fname = copy.copy(index)
fp_out = os.path.join(gdb, 'analysis', fname)
print('FILE PATH OUT: ', fp_out)

# # b) Once arguments are confirmed, uncomment and run
arcpy.Clip_analysis(fp_in, fp_clip, fp_out)

In [23]:
# Cell 17) Set values to DataFrame
df.loc[index,'DATA_LOCATION'] = fp_out

# ADD DATA INFO
df.loc[index, 'DATE'] = '20221004'
df.loc[index, 'SOURCE'] = 'McMillen Jacobs'
df.loc[index, 'ABSTRACT'] = 'Contours derived from the USGS National Elevation Dataset 1/3 arc' \
                            ' second n45w117 20220309.  DEM was smoothed with a gaussian filter' \
                            'prior to generating contours.  Contours at 100m elevation band interval.'

In [ ]:
# Cell 18) Clean up map
# remove contours_100m_4corners

In [8]:
# Cell 19) Inspect dataframe prior to saving.
print(df)
print('\nABSTRACT: \n{}'.format(df.loc[index, 'ABSTRACT']))

In [29]:
# 20) Save udpated CSV 
# NOTE that usually I save directly to original csv - but for repeating, retain original
# Saved to notebooks_tutorial directory.  Open and take a look.
fp_csv_updated = os.path.join(user_git_dir,r'four_corners_morels_inv_updated.csv')
df.to_csv(fp_csv_updated)